In [ ]:
import psycopg2
import pandas as pd
from utils.utils import get_postgres_connection


def read_financial_metrics():
    """Simple function to read financial_metrics table and return as pandas DataFrame"""
    conn = get_postgres_connection()
    query = "SELECT * FROM raw.financial_metrics"
    df = pd.read_sql_query(query, conn)
    conn.close()
    return df


df = read_financial_metrics()
print(f"Read {len(df)} records from financial_metrics table")
print(df.head())


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x1079a5490>>
Traceback (most recent call last):
  File "/Users/aleksamihajlovic/Documents/naro-index-advisor/etl-service/.venv/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 781, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


In [32]:
df.columns

Index(['symbol', 'date', 'fiscal_year', 'period', 'reported_currency',
       'gross_profit_margin', 'ebit_margin', 'ebitda_margin',
       'operating_profit_margin', 'pretax_profit_margin',
       'continuous_operations_profit_margin', 'net_profit_margin',
       'bottom_line_profit_margin', 'current_ratio', 'quick_ratio',
       'solvency_ratio', 'cash_ratio', 'receivables_turnover',
       'payables_turnover', 'inventory_turnover', 'fixed_asset_turnover',
       'asset_turnover', 'working_capital_turnover_ratio',
       'price_to_earnings_ratio', 'price_to_earnings_growth_ratio',
       'forward_price_to_earnings_growth_ratio', 'price_to_book_ratio',
       'price_to_sales_ratio', 'price_to_free_cash_flow_ratio',
       'price_to_operating_cash_flow_ratio', 'price_to_fair_value',
       'debt_to_assets_ratio', 'debt_to_equity_ratio', 'debt_to_capital_ratio',
       'long_term_debt_to_capital_ratio', 'financial_leverage_ratio',
       'debt_to_market_cap', 'operating_cash_flow_ratio'

In [31]:
import pandas as pd
import numpy as np
from IPython.display import display_html

def _safe_numeric_series(s: pd.Series) -> pd.Series:
    """Keep only finite numeric values."""
    s = pd.to_numeric(s, errors="coerce")
    s = s.replace([np.inf, -np.inf], np.nan).dropna()
    return s

def _summary_with_custom_percentiles(s: pd.Series, percentiles=None) -> pd.DataFrame:
    if percentiles is None:
        # include 5th, 95th and deciles 10..90
        percentiles = [0.001, 0.01,0.05] + [i/100 for i in range(10, 100, 10)] + [0.95, 0.99, 0.999]
    desc = s.describe(percentiles=percentiles)
    out = desc.reset_index()
    out.columns = ["Statistic", "Value"]
    return out

def _bucket_counts_qcut(s: pd.Series, q=10) -> pd.DataFrame:
    """Counts per percentile bucket, safe against duplicates and constants."""
    n = len(s)
    if n == 0:
        return pd.DataFrame({"Percentile Bucket": [], "Count": []})
    if s.nunique(dropna=True) < 2:
        return pd.DataFrame({"Percentile Bucket": ["All values equal"], "Count": [n]})

    try:
        buckets = pd.qcut(s, q=q, labels=[f"{i*100//q}-{(i+1)*100//q}%" for i in range(q)], duplicates="drop")
        counts = buckets.value_counts().sort_index().reset_index()
        counts.columns = ["Percentile Bucket", "Count"]
        return counts
    except Exception:
        qs = np.linspace(0, 1, q + 1)
        edges = np.unique(s.quantile(qs).values)
        if len(edges) < 2:
            return pd.DataFrame({"Percentile Bucket": ["All values equal"], "Count": [n]})
        labels = [f"{int(qs[i]*100)}-{int(qs[i+1]*100)}%" for i in range(len(edges)-1)]
        buckets = pd.cut(s, bins=edges, include_lowest=True, labels=labels, duplicates="drop")
        counts = buckets.value_counts().sort_index().reset_index()
        counts.columns = ["Percentile Bucket", "Count"]
        return counts

def _display_side_by_side(dfs: list, titles: list):
    html = ""
    for df, title in zip(dfs, titles):
        html += (
            "<div style='display:inline-block; padding-right:30px; vertical-align:top;'>"
            f"<h3 style='margin:4px 0 8px 0;'>{title}</h3>"
            f"{df.to_html(index=False)}"
            "</div>"
        )
    display_html(html, raw=True)

def describe_all_numeric_with_buckets(df: pd.DataFrame, q=10, percentiles=None, max_cols=None):
    """
    For each numeric column:
      - show summary with custom percentiles (5th, 10..90, 95th by default)
      - show counts per percentile bucket
    """
    num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    if max_cols is not None:
        num_cols = num_cols[:max_cols]

    if not num_cols:
        print("No numeric columns found.")
        return

    for col in num_cols:
        s = _safe_numeric_series(df[col])
        if s.empty:
            print(f"📭 Column: {col} — skipped (no finite numeric values).")
            continue

        stats = _summary_with_custom_percentiles(s, percentiles=percentiles)
        buckets = _bucket_counts_qcut(s, q=q)

        print(f"📊 Column: {col} (n={len(s)})")
        _display_side_by_side(
            [stats, buckets],
            ["Summary Statistics", f"Counts per Percentile Bucket ({q} buckets)"]
        )
        print()  # spacing


# ---- Run it ----
# This will include 5th, 10..90, 95th percentiles + decile bucket counts
describe_all_numeric_with_buckets(df, q=10)


📊 Column: gross_profit_margin (n=1557417)


Statistic,Value
count,1557417.000000
mean,-2436.884422
std,2346429.840695
min,-2882786194.286195
0.1%,-138.667394
1%,-1.857993
5%,0.000000
10%,0.000000
20%,0.077175
30%,0.157231



📊 Column: ebit_margin (n=1557413)


Statistic,Value
count,1557413.000000
mean,-5928.403458
std,4323168.063856
min,-3882786195.286195
0.1%,-4999.881298
1%,-56.557902
5%,-1.692778
10%,-0.313041
20%,-0.018701
30%,0.000000



📊 Column: ebitda_margin (n=1557391)


...

Statistic,Value
count,1557404.000000
mean,2518.682675
std,5791877.164853
min,-3448599150.000000
0.1%,-128.162378
1%,-9.522640
5%,-0.679332
10%,0.000000
20%,0.589197
30%,0.828136



📊 Column: effective_tax_rate (n=1556820)


Statistic,Value
count,1556820.000000
mean,-1584.142644
std,1815550.187148
min,-2263290476.190476
0.1%,-26.008292
1%,-2.076302
5%,-0.266929
10%,-0.049229
20%,0.000000
30%,0.000000



📊 Column: enterprise_value_multiple (n=1471242)


Statistic,Value
count,1471242.000000
mean,-37826.837472
std,27179300.354871
min,-9700800029.078014
0.1%,-225953.778409
1%,-3003.765372
5%,-235.870537
10%,-70.021770
20%,-10.006357
30%,0.068124


#### PIPELINE
#### PIPELINE
#### PIPELINE
#### PIPELINE

✅ Table clean.financial_metrics_perc recreated and populated.


In [51]:
import polars as pl
import numpy as np
import io
import pandas as pd
import psycopg2
from sqlalchemy import create_engine, text
from sqlalchemy.engine import Engine
from utils.utils import get_database_url, get_postgres_connection

# --------------------------
# Config & Setup
# --------------------------
DATABASE_URL = get_database_url()
target_table = "clean.financial_metrics_perc"
staging_schema = "stage"

# Metrics to process
metrics = [
    "gross_profit_margin",
    "ebit_margin",
    "ebitda_margin",
    "operating_profit_margin",
    "pretax_profit_margin",
    "continuous_operations_profit_margin",
    "net_profit_margin",
    "bottom_line_profit_margin",
    "current_ratio",
    "quick_ratio",
    "solvency_ratio",
    "cash_ratio",
    "receivables_turnover",
    "payables_turnover",
    "inventory_turnover",
    "fixed_asset_turnover",
    "asset_turnover",
    "working_capital_turnover_ratio",
    "price_to_earnings_ratio",
    "price_to_earnings_growth_ratio",
    "forward_price_to_earnings_growth_ratio",
    "price_to_book_ratio",
    "price_to_sales_ratio",
    "price_to_free_cash_flow_ratio",
    "price_to_operating_cash_flow_ratio",
    "price_to_fair_value",
    "debt_to_assets_ratio",
    "debt_to_equity_ratio",
    "debt_to_capital_ratio",
    "long_term_debt_to_capital_ratio",
    "financial_leverage_ratio",
    "debt_to_market_cap",
    "operating_cash_flow_ratio",
    "operating_cash_flow_sales_ratio",
    "free_cash_flow_operating_cash_flow_ratio",
    "debt_service_coverage_ratio",
    "interest_coverage_ratio",
    "short_term_operating_cash_flow_coverage_ratio",
    "operating_cash_flow_coverage_ratio",
    "capital_expenditure_coverage_ratio",
    "dividend_paid_and_capex_coverage_ratio",
    "dividend_payout_ratio",
    "dividend_yield",
    "dividend_yield_percentage",
    "dividend_per_share",
    "revenue_per_share",
    "net_income_per_share",
    "interest_debt_per_share",
    "cash_per_share",
    "book_value_per_share",
    "tangible_book_value_per_share",
    "shareholders_equity_per_share",
    "operating_cash_flow_per_share",
    "capex_per_share",
    "free_cash_flow_per_share",
    "net_income_per_ebt",
    "ebt_per_ebit",
    "effective_tax_rate",
    "enterprise_value_multiple"

]

# Columns always kept in final table
identity_columns = [
    'symbol',
    'date',
    'fiscal_year',
    'period',
    'reported_currency'
]

# --------------------------
# 1. Create/Populate Target Table
# --------------------------
print("📦 Recreating clean.financial_metrics_perc...")
engine: Engine = create_engine(DATABASE_URL)
with engine.begin() as conn:
    conn.execute(text("DROP TABLE IF EXISTS clean.financial_metrics_perc"))
    conn.execute(text(f"""
        CREATE TABLE clean.financial_metrics_perc AS
        SELECT {', '.join(identity_columns)}
        FROM raw.financial_metrics
        WHERE FALSE
    """))
    conn.execute(text(f"""
        INSERT INTO clean.financial_metrics_perc ({', '.join(identity_columns)})
        SELECT {', '.join(identity_columns)}
        FROM raw.financial_metrics
    """))
print("✅ clean.financial_metrics_perc initialized.")

# --------------------------
# 2. Load Full Raw Dataset
# --------------------------
def read_financial_metrics():
    conn = get_postgres_connection()
    cur = conn.cursor()
    buf = io.BytesIO()
    cur.copy_expert("COPY raw.financial_metrics TO STDOUT WITH CSV HEADER", buf)
    buf.seek(0)
    df = pl.read_csv(buf)
    cur.close()
    conn.close()
    return df

df_raw = read_financial_metrics()

# --------------------------
# 3. Percentile Processing + Insertion Loop
# --------------------------
percentile_levels = [0.01, 0.10, 0.20, 0.30, 0.40, 0.50, 0.60, 0.70, 0.80, 0.90, 0.99]
labels = ["<1%", "10%", "20%", "30%", "40%", "50%", "60%", "70%", "80%", "90%", "99%", ">99%"]

pg_conn = psycopg2.connect(DATABASE_URL.replace("postgresql+psycopg2", "postgresql"))
cursor = pg_conn.cursor()

for metric in metrics:
    perc_column = f"{metric}_perc"
    staging_table = f"tmp_{perc_column}"

    print(f"\n📊 Processing: {metric} → {perc_column}")

    # --------------------------
    # Create Percentile Buckets
    # --------------------------
    df_metric = df_raw.select(identity_columns + [metric]).filter(pl.col(metric).is_not_null())

    cutoffs = df_metric.select([
        pl.col(metric).quantile(q, "nearest").alias(f"p{int(q*100):02d}")
        for q in percentile_levels
    ]).row(0)

    bins = [-np.inf] + list(cutoffs) + [np.inf]
    bounds = list(zip(bins[:-1], bins[1:]))

    mapping_df = pl.DataFrame({
        "label": labels,
        "low": [bounds[i][0] for i in range(len(bounds))],
        "high": [bounds[i][1] for i in range(len(bounds))]
    }).with_columns([
        pl.when(pl.col("low") == float("-inf"))
        .then(pl.lit("-∞"))
        .otherwise(pl.col("low").round(2).cast(pl.Utf8))
        .alias("low_fmt"),

        pl.when(pl.col("high") == float("inf"))
        .then(pl.lit("+∞"))
        .otherwise(pl.col("high").round(2).cast(pl.Utf8))
        .alias("high_fmt")
    ]).with_columns([
        (pl.col("label") + " (" + pl.col("low_fmt") + " – " + pl.col("high_fmt") + ")")
        .alias("bracket_display")
    ])

    df_with_perc = (
        df_metric.lazy()
        .join(mapping_df.lazy(), how="cross")
        .filter(
            (pl.col(metric) >= pl.col("low")) & (pl.col(metric) < pl.col("high")) |
            ((pl.col("label") == ">99%") & (pl.col(metric) == pl.col("high")))
        )
        .select([
            "symbol", "date",
            pl.col("bracket_display").alias(perc_column)
        ])
        .collect()
    )

    # --------------------------
    # Write to Staging Table
    # --------------------------
    df_pd = df_with_perc.to_pandas()
    df_pd["date"] = pd.to_datetime(df_pd["date"])

    cursor.execute(f"SET search_path TO {staging_schema}")
    cursor.execute(f"DROP TABLE IF EXISTS {staging_table}")
    cursor.execute(f"""
        CREATE TABLE {staging_table} (
            symbol TEXT,
            date DATE,
            {perc_column} TEXT
        )
    """)
    pg_conn.commit()

    output = io.StringIO()
    df_pd.to_csv(output, sep="\t", index=False, header=False, na_rep='\\N')
    output.seek(0)

    cursor.copy_from(output, staging_table, sep="\t", null='\\N')
    pg_conn.commit()
    print("📥 Uploaded to staging table.")

    # --------------------------
    # Ensure Target Column Exists
    # --------------------------
    with engine.begin() as conn:
        conn.execute(text(f"""
            ALTER TABLE {target_table}
            ADD COLUMN IF NOT EXISTS {perc_column} VARCHAR(30);
        """))

    # --------------------------
    # Merge into Final Table
    # --------------------------
    with engine.begin() as conn:
        conn.execute(text(f"""
            UPDATE {target_table} t
            SET {perc_column} = s.{perc_column}
            FROM {staging_schema}.{staging_table} s
            WHERE t.symbol = s.symbol
              AND t.date = s.date::date
        """))

    print(f"✅ Column {perc_column} updated in target table")

    # --------------------------
    # Cleanup staging table
    # --------------------------

    cursor.execute(f"DROP TABLE IF EXISTS {staging_schema}.{staging_table}")
    pg_conn.commit()
    print(f"🧹 Dropped staging table: {staging_schema}.{staging_table}")


📦 Recreating clean.financial_metrics_perc...
✅ clean.financial_metrics_perc initialized.

📊 Processing: gross_profit_margin → gross_profit_margin_perc
📥 Uploaded to staging table.
✅ Column gross_profit_margin_perc updated in target table
🧹 Dropped staging table: stage.tmp_gross_profit_margin_perc

📊 Processing: ebit_margin → ebit_margin_perc
📥 Uploaded to staging table.
✅ Column ebit_margin_perc updated in target table
🧹 Dropped staging table: stage.tmp_ebit_margin_perc

📊 Processing: ebitda_margin → ebitda_margin_perc
📥 Uploaded to staging table.
✅ Column ebitda_margin_perc updated in target table
🧹 Dropped staging table: stage.tmp_ebitda_margin_perc

📊 Processing: operating_profit_margin → operating_profit_margin_perc
📥 Uploaded to staging table.
✅ Column operating_profit_margin_perc updated in target table
🧹 Dropped staging table: stage.tmp_operating_profit_margin_perc

📊 Processing: pretax_profit_margin → pretax_profit_margin_perc
📥 Uploaded to staging table.
✅ Column pretax_profit

In [1]:
import io
import numpy as np
import pandas as pd
import polars as pl
import psycopg2
from sqlalchemy import create_engine, text
from sqlalchemy.engine import Engine
from utils.utils import get_database_url, get_postgres_connection

# --------------------------
# Config
# --------------------------
DATABASE_URL = get_database_url()
engine: Engine = create_engine(DATABASE_URL)
target_table = "clean.financial_metrics_perc"
staging_schema = "stage"

identity_columns = ["symbol", "date", "fiscal_year", "period", "reported_currency"]

metrics = [
    "gross_profit_margin", "ebit_margin", "ebitda_margin",
    "operating_profit_margin", "pretax_profit_margin",
    "continuous_operations_profit_margin", "net_profit_margin",
    "bottom_line_profit_margin", "current_ratio", "quick_ratio",
    "solvency_ratio", "cash_ratio", "receivables_turnover",
    "payables_turnover", "inventory_turnover", "fixed_asset_turnover",
    "asset_turnover", "working_capital_turnover_ratio",
    "price_to_earnings_ratio", "price_to_earnings_growth_ratio",
    "forward_price_to_earnings_growth_ratio", "price_to_book_ratio",
    "price_to_sales_ratio", "price_to_free_cash_flow_ratio",
    "price_to_operating_cash_flow_ratio", "price_to_fair_value",
    "debt_to_assets_ratio", "debt_to_equity_ratio",
    "debt_to_capital_ratio", "long_term_debt_to_capital_ratio",
    "financial_leverage_ratio", "debt_to_market_cap",
    "operating_cash_flow_ratio", "operating_cash_flow_sales_ratio",
    "free_cash_flow_operating_cash_flow_ratio",
    "debt_service_coverage_ratio", "interest_coverage_ratio",
    "short_term_operating_cash_flow_coverage_ratio",
    "operating_cash_flow_coverage_ratio",
    "capital_expenditure_coverage_ratio",
    "dividend_paid_and_capex_coverage_ratio",
    "dividend_payout_ratio", "dividend_yield", "dividend_yield_percentage",
    "dividend_per_share", "revenue_per_share", "net_income_per_share",
    "interest_debt_per_share", "cash_per_share", "book_value_per_share",
    "tangible_book_value_per_share", "shareholders_equity_per_share",
    "operating_cash_flow_per_share", "capex_per_share",
    "free_cash_flow_per_share", "net_income_per_ebt", "ebt_per_ebit",
    "effective_tax_rate", "enterprise_value_multiple"
]

percentile_levels = [0.01,0.10,0.20,0.30,0.40,0.50,0.60,0.70,0.80,0.90,0.99]
labels = ["<1%","10%","20%","30%","40%","50%","60%","70%","80%","90%","99%",">99%"]

BATCH_SIZE = 10

# --------------------------
# Helpers
# --------------------------
def read_financial_metrics() -> pl.DataFrame:
    conn = get_postgres_connection()
    cur = conn.cursor()
    buf = io.BytesIO()
    cur.copy_expert("COPY raw.financial_metrics TO STDOUT WITH CSV HEADER", buf)
    buf.seek(0)
    df = pl.read_csv(buf, infer_schema_length=10000)
    cur.close()
    conn.close()
    return df.with_columns([
        pl.col("symbol").cast(pl.Utf8),
        pl.col("date").str.strptime(pl.Date, strict=False),
        pl.col("fiscal_year").cast(pl.Int64),
        pl.col("period").cast(pl.Utf8),
        pl.col("reported_currency").cast(pl.Utf8),
    ])

def bucketize_metric(df: pl.DataFrame, metric: str) -> pl.DataFrame:
    non_null = df.select(identity_columns + [metric]).filter(pl.col(metric).is_not_null())
    if non_null.height == 0:
        return pl.DataFrame({"symbol": [], "date": [], f"{metric}_perc": []})
    qs = non_null.select([
        pl.col(metric).quantile(q, "nearest").alias(f"p{int(q*100):02d}")
        for q in percentile_levels
    ]).row(0)
    bins = [-np.inf] + list(qs) + [np.inf]
    lows, highs = bins[:-1], bins[1:]
    def fmt(x): return "-∞" if x == float("-inf") else "+∞" if x == float("inf") else f"{x:.2f}"
    bracket_display = [f"{lab} ({fmt(lo)} – {fmt(hi)})" for lab, lo, hi in zip(labels, lows, highs)]
    mapping_df = pl.DataFrame({"_low": lows, "_high": highs, "_label": bracket_display})
    return (
        non_null.lazy()
        .join(mapping_df.lazy(), how="cross")
        .filter((pl.col(metric) >= pl.col("_low")) & (pl.col(metric) < pl.col("_high")))
        .select(["symbol","date",pl.col("_label").alias(f"{metric}_perc")])
        .collect()
    )

# --------------------------
# 1) Read raw + identities
# --------------------------
print("📥 Reading raw.financial_metrics …")
df_raw = read_financial_metrics()
df_base = df_raw.select(identity_columns).unique()

# --------------------------
# 2) Drop + recreate target
# --------------------------
print("🧨 Dropping + recreating target table …")
with engine.begin() as conn:
    conn.execute(text(f"DROP TABLE IF EXISTS {target_table}"))
    ddl = f"""
        CREATE TABLE {target_table} (
            symbol TEXT,
            date DATE,
            fiscal_year INTEGER,
            period TEXT,
            reported_currency TEXT
        ) WITH (fillfactor=100);
    """
    conn.execute(text(ddl))

# Insert identities once
pg_conn = psycopg2.connect(DATABASE_URL.replace("postgresql+psycopg2","postgresql"))
cur = pg_conn.cursor()
buf = io.StringIO()
df_base.to_pandas().to_csv(buf, sep="\t", index=False, header=False, na_rep="\\N")
buf.seek(0)
cols_sql = ", ".join([f'"{c}"' for c in identity_columns])
cur.copy_expert(f"COPY {target_table} ({cols_sql}) FROM STDIN WITH (FORMAT text, DELIMITER E'\\t', NULL '\\N')", buf)
pg_conn.commit()

# --------------------------
# 3) Process metrics in batches
# --------------------------
for i in range(0, len(metrics), BATCH_SIZE):
    batch = metrics[i:i+BATCH_SIZE]
    print(f"📊 Processing batch {i//BATCH_SIZE+1}: {batch}")

    batch_df = df_base
    for m in batch:
        df_m = bucketize_metric(df_raw, m)
        if df_m.height > 0:
            batch_df = batch_df.join(df_m, on=["symbol","date"], how="left")

    pdf = batch_df.to_pandas()
    pdf["date"] = pd.to_datetime(pdf["date"]).dt.date

    # Create staging
    staging = f"{staging_schema}.tmp_batch_{i}"
    cur.execute(f"DROP TABLE IF EXISTS {staging}")
    cur.execute(f'CREATE TABLE {staging} (symbol TEXT, date DATE, ' +
                ", ".join([f'"{m}_perc" TEXT' for m in batch]) + ")")
    pg_conn.commit()

    # COPY batch into staging
    buf = io.StringIO()
    pdf[["symbol","date"] + [f"{m}_perc" for m in batch]].to_csv(
        buf, sep="\t", index=False, header=False, na_rep="\\N"
    )
    buf.seek(0)
    cols = ["symbol","date"] + [f"{m}_perc" for m in batch]
    cols_sql = ", ".join([f'"{c}"' for c in cols])
    cur.copy_expert(f"COPY {staging} ({cols_sql}) FROM STDIN WITH (FORMAT text, DELIMITER E'\\t', NULL '\\N')", buf)
    pg_conn.commit()

    # Ensure target has the columns
    with engine.begin() as conn:
        for m in batch:
            conn.execute(text(f'ALTER TABLE {target_table} ADD COLUMN IF NOT EXISTS "{m}_perc" TEXT'))

        set_clause = ", ".join([f'"{m}_perc" = s."{m}_perc"' for m in batch])
        conn.execute(text(f"""
            UPDATE {target_table} t
            SET {set_clause}
            FROM {staging} s
            WHERE t.symbol = s.symbol
              AND t.date = s.date
        """))

    cur.execute(f"DROP TABLE IF EXISTS {staging}")
    pg_conn.commit()
    print(f"✅ Batch {i//BATCH_SIZE+1} merged.")

cur.close()
pg_conn.close()
print("🎉 Done — all batches merged into final table without overwriting.")


📥 Reading raw.financial_metrics …
🧨 Dropping + recreating target table …
📊 Processing batch 1: ['gross_profit_margin', 'ebit_margin', 'ebitda_margin', 'operating_profit_margin', 'pretax_profit_margin', 'continuous_operations_profit_margin', 'net_profit_margin', 'bottom_line_profit_margin', 'current_ratio', 'quick_ratio']
✅ Batch 1 merged.
📊 Processing batch 2: ['solvency_ratio', 'cash_ratio', 'receivables_turnover', 'payables_turnover', 'inventory_turnover', 'fixed_asset_turnover', 'asset_turnover', 'working_capital_turnover_ratio', 'price_to_earnings_ratio', 'price_to_earnings_growth_ratio']
✅ Batch 2 merged.
📊 Processing batch 3: ['forward_price_to_earnings_growth_ratio', 'price_to_book_ratio', 'price_to_sales_ratio', 'price_to_free_cash_flow_ratio', 'price_to_operating_cash_flow_ratio', 'price_to_fair_value', 'debt_to_assets_ratio', 'debt_to_equity_ratio', 'debt_to_capital_ratio', 'long_term_debt_to_capital_ratio']
✅ Batch 3 merged.
📊 Processing batch 4: ['financial_leverage_ratio',